In [69]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [73]:
# load the data to pandas dataset
news_data=pd.read_csv('/content/drive/MyDrive/Machine Learning/Datasets/Fake_News/train.csv')

In [74]:
# first 5 rows of dataframe
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


0=real news
1=fake news

In [75]:
news_data.shape

(20800, 5)

In [76]:
# checking the null values
# news_data['author'].isnull().sum()
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [77]:
# replacing the missing values with null string
news_data=news_data.fillna('')

In [78]:
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [79]:
# we gonna make a new column in which we will merge the column author and news title
# new column will be content

news_data['content']=news_data['author']+' '+news_data['title']
news_data

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [80]:
# seperating the content and label because doing processing on text can be very costly
X=news_data.drop(columns='label',axis=1)
Y=news_data['label']

In [81]:
print(X.shape)
print(Y)

(20800, 5)
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [82]:
# Stemming
# stemming is trhe process  of reducing the words to its keyword
# eg enjoying=enjoy

port_stem=PorterStemmer()

In [83]:
# create a function which do stemming
def stemming(content):
  # content = ' '.join(content)
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)#iterate through every content of content col and replace the non alphabetic with space r.sub(pattern,replacement,string)
  stemmed_content=stemmed_content.lower()#converted all the stemmeedd content to lower case
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#stemmed  every word in content except stopwords
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [84]:
# print(news_data['content'].apply(type).unique())
# can know the data type of list


In [85]:
news_data['content'] = news_data['content'].apply(stemming)

In [86]:
news_data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [87]:
X=news_data['content']
Y=news_data['label']

In [88]:
# feature extraction
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(X)


In [89]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 2483)	0.3676519686797209
  (0, 7692)	0.24785219520671603
  (0, 8630)	0.29212514087043684
  (0, 2959)	0.2468450128533713
  (0, 13473)	0.2565896679337957
  (0, 4973)	0.233316966909351
  (0, 267)	0.27010124977708766
  (0, 3792)	0.2705332480845492
  (0, 7005)	0.21874169089359144
  (0, 8909)	0.3635963806326075
  (0, 3600)	0.3598939188262559
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 16799)	0.30071745655510157
  (1, 1497)	0.2939891562094648
  (1, 2813)	0.19094574062359204
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  (2, 15611)	0.41544962664721613
  (2, 2943)	0.3179886800654691
  :	:
  (20797, 1287)	0.3353805680413986
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 14967)	0.3115945315488075
  (20797, 12138)	0.24778257724396505
  (20797, 9518)	0.295420

In [90]:
# splitting the datset into training and test data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

training the model LOgistic Regression

In [91]:
model=LogisticRegression()

In [92]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation of the model

In [93]:
# prediction accuracy on training data

X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [94]:
print("Accuracy of our Model on training data is:",training_data_accuracy)

Accuracy of our Model on training data is: 0.9873798076923077


In [95]:
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [96]:
print("Accuracy on the testing data is: ",testing_data_accuracy)

Accuracy on the testing data is:  0.9774038461538461


In [104]:
# taking the user input
# Taking user input
while True:
    choice = input("Enter 1 for continue and 0 for stopping")
    print("You entered:", choice)  # Debugging line to check the value of choice
    if choice == "0":
        break
    else:
        user_input = input("Enter the news: ")
        # Preprocess the user input (you can customize this preprocessing)
        # For simplicity, let's assume lowercase and tokenization here
        preprocessed_input = user_input.lower()  # Tokenization

        # Transform the preprocessed input using the same vectorizer
        input_vect = vectorizer.transform([" ".join(preprocessed_input)])

        # Predict using the trained model
        prediction = model.predict(input_vect)

        if prediction[0] == 0:
            print("Real News")
        else:
            print("Fake News")





Enter 1 for continue and 0 for stopping1
You entered: 1
Enter the news: Workers at an Ultium Cells plant in Ohio are fighting for fair wages and safety on the job. What happens there could help set the tone for similar joint-venture plants across the country.
Fake News
Enter 1 for continue and 0 for stoppingTHERE’S NO SUCH THING AS A ‘UNION BOSS’—WHAT LABOR REPORTERS GET WRONG Kim Kelly, Sarah Jaffe, Abigail Higgins, and Braden Campbell dig into the ins and outs of labor reporting, how unions actually work, and what media gets wrong about the workers’ struggle over and over again.
You entered: THERE’S NO SUCH THING AS A ‘UNION BOSS’—WHAT LABOR REPORTERS GET WRONG Kim Kelly, Sarah Jaffe, Abigail Higgins, and Braden Campbell dig into the ins and outs of labor reporting, how unions actually work, and what media gets wrong about the workers’ struggle over and over again.
Enter the news: THERE’S NO SUCH THING AS A ‘UNION BOSS’—WHAT LABOR REPORTERS GET WRONG Kim Kelly, Sarah Jaffe, Abigail H